In [ ]:
chosen_collection = "MSMARCO"

In [ ]:
import gdown
import ir_datasets
import pandas as pd
import os

if chosen_collection not in ["vaswani", "MSMARCO"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'MSMARCO']")

if chosen_collection == "MSMARCO":
    url_collection = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
    
    if not os.path.exists('../MSMARCO.tsv'):
        gdown.download(url_collection, 'MSMARCO.tsv', quiet=False)

    #df = pd.read_csv('../collection.tsv', sep='\t', names=['doc_id', 'text'])
   

elif chosen_collection == "vaswani":
    vaswani_dataset = ir_datasets.load("vaswani")
    docs = list(vaswani_dataset.docs_iter())
    df = pd.DataFrame(docs)
    df['doc_id'] = (df['doc_id'].astype(int) - 1).astype(str)
    # rimuovi i \n da ogni documento
    df['text'] = df['text'].str.replace('\n', ' ')
    df.to_csv('vaswani.tsv', sep='\t', header=False, index=False)

In [ ]:
# from collections import Counter
# import string
# import nltk
# import re
# from tqdm import trange

# nltk.download("stopwords", quiet=True)
# STOPWORDS = set(nltk.corpus.stopwords.words("english"))
# STEMMER = nltk.stem.PorterStemmer()

# def analyze_collection(documents):
#     char_counter = Counter()
#     stopword_counter = Counter()
#     contains_acronyms = 0
#     total_words = 0
#     total_docs = len(documents)

#     for doc in trange(total_docs, desc="Processing documents"):
#         # Assicurati che 'doc' sia una stringa e non un indice
#         document_text = documents.iloc[doc]  # Ottieni il testo del documento tramite l'indice
        
#         # Conteggio dei caratteri
#         char_counter.update(list(document_text))  # Conta i caratteri del testo
        
#         # Stopword e parole totali
#         words = document_text.lower().split()
#         total_words += len(words)
#         stopword_counter.update(word for word in words if word in STOPWORDS)
        
#         # Controllo acronimi
#         if re.search(r"\b\w+\.\w+\.\b", document_text):  # Cerca pattern tipo "U.S.A."
#             contains_acronyms += 1

#     # Filtra solo i caratteri speciali
#     special_chars = {ch: count for ch, count in char_counter.items() if ch in string.punctuation or not ch.isalnum()}

#     # Calcola le statistiche
#     percent_documents_with_acronyms = (contains_acronyms / total_docs) * 100
#     percent_special_chars = (sum(special_chars.values()) / sum(char_counter.values())) * 100
#     percent_stopwords = (sum(stopword_counter.values()) / total_words) * 100

#     return {
#         "special_chars": special_chars,
#         "contains_acronyms": contains_acronyms,
#         "percent_documents_with_acronyms": percent_documents_with_acronyms,
#         "percent_special_chars": percent_special_chars,
#         "percent_stopwords": percent_stopwords
#     }

# # Esegui l'analisi sulla colonna 'text' di df
# result = analyze_collection(df['text'])

# # Stampa i risultati
# print("Special characters:", result["special_chars"])
# print(f"Documents with acronyms: {result['contains_acronyms']} ({result['percent_documents_with_acronyms']:.2f}%)")
# print(f"Percentage of special characters: {result['percent_special_chars']:.2f}%")
# print(f"Percentage of stopwords: {result['percent_stopwords']:.2f}%")

In [ ]:
import re
import string
import nltk
import Stemmer
STEMMERNEW = Stemmer.Stemmer('english')



nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
#STEMMER = nltk.stem.PorterStemmer()


def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand and special chars
    s = re.sub(r"[‘’´“”–-]", "'", s.replace("&", " and ")) # this replaces & with 'and' and normalises quotes
    # acronyms
    #s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # this removes dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = re.sub(r"\s+", " ", s).strip()
    # tokenisation
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # stemming
    #tokens = [STEMMER.stem(t) for t in tokens]
    
    return STEMMERNEW.stemWords(tokens)

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import Stemmer

# Stemmer di PyStemmer (Porter2)
pystemmer = Stemmer.Stemmer('english')

# Stemmer Porter originale
nltk_porter = PorterStemmer()

# Stemmer Porter2 di NLTK
nltk_snowball = SnowballStemmer('english')

# Parola di esempio
word = "general"

print("PyStemmer:", pystemmer.stemWord(word))
print("NLTK Porter:", nltk_porter.stem(word))
print("NLTK Snowball (Porter2):", nltk_snowball.stem(word))


In [ ]:
import re
import string
from nltk.stem import PorterStemmer
import nltk



nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = PorterStemmer()


def preprocess_old(s):
    # lowercasing
    s = s.lower()
    # ampersand and special chars
    s = re.sub(r"[‘’´“”–-]", "'", s.replace("&", " and ")) # this replaces & with 'and' and normalises quotes
    # acronyms
    #s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # this removes dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = re.sub(r"\s+", " ", s).strip()
    # tokenisation
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # stemming
    tokens = [STEMMER.stem(t) for t in tokens]
    
    return tokens

In [ ]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [ ]:
import pickle
import humanize
import os

def print_pickled_size(var_name, var):
    # If the 'tmp' directory does not exist, we first create it
    os.makedirs('./tmp', exist_ok=True)
    with open(f"./tmp/{var_name}.pickle", 'wb') as f:
        pickle.dump(var, f)
    print(f'{var_name} requires {humanize.naturalsize(os.path.getsize(f"./tmp/{var_name}.pickle"))}')
    os.remove(f"./tmp/{var_name}.pickle")
    os.removedirs('./tmp')


def vbyte_encode(number):
    bytes_list = bytearray()
    while True:
        byte = number & 0x7F # Prendi i 7 bit meno significativi -> 0111 1111 = 0x7F
        number >>= 7 # Shifta a destra di 7 bit
        if number:
            bytes_list.append(byte) # Aggiungo i 7 bit al risultato
        else:
            bytes_list.append(0x80 | byte) # Aggiungo i 7 bit con il bit di continuazione, 0x80 = 1000 0000
            break
    return bytes(bytes_list)

def vbyte_decode(bytes_seq):
    number = 0
    for i, byte in enumerate(bytes_seq):
        number |= (byte & 0x7F) << (7 * i)
        if byte & 0x80:
            break
    return number

def decode_concatenated_vbyte(encoded_bytes):
    decoded_numbers = []
    current_number = 0
    shift_amount = 0
    
    for byte in encoded_bytes:
        if byte & 0x80:  # Bit di continuazione trovato, fine del numero
            current_number |= (byte & 0x7F) << shift_amount
            decoded_numbers.append(current_number)
            current_number = 0
            shift_amount = 0
        else:  # Continuo a comporre il numero
            current_number |= (byte & 0x7F) << shift_amount
            shift_amount += 7
    
    return decoded_numbers

In [ ]:
from collections import Counter
from tqdm.auto import tqdm
import pandas as pd

@profile
def build_index(file_path, batchsize=10000):
    """
    Costruisce un indice invertito da un file TSV senza intestazione elaborato in batch,
    mostrando una barra di avanzamento completa.
    
    Args:
        file_path (str): Percorso al file TSV senza intestazione.
        batchsize (int): Numero di righe da elaborare per batch (default: 10000).
        
    Returns:
        tuple: lexicon (vocabolario), indici invertiti (docids, freqs), doc_index (documenti e lunghezze), stats (statistiche globali).
    """
    lexicon = {}
    doc_index = []
    inv_d, inv_f = {}, {}
    termid = 0

    num_docs = 0
    total_dl = 0

    # Calcolare il numero totale di righe/documenti
    with open(file_path, 'r', encoding='utf-8') as f:
        total_docs = sum(1 for _ in f)
    
    # Barra di avanzamento con totale definito
    progress_bar = tqdm(desc="Indexing", total=total_docs, unit="doc", dynamic_ncols=True)
    
    # Leggere il file TSV in batch
    for df in pd.read_csv(file_path, sep='\t', header=None, names=['doc_id', 'text'], chunksize=batchsize):
        for docid, (doc_id, doc_text) in enumerate(zip(df['doc_id'], df['text'])):
            tokens = preprocess(doc_text)
            token_tf = Counter(tokens)
            #doclen = sum(token_tf.values())
            for token, tf in token_tf.items():
                if token not in lexicon:
                    lexicon[token] = [termid, 0, 0]
                    inv_d[termid], inv_f[termid] = [], []
                    termid += 1
                token_id = lexicon[token][0]  # prendo il termid
                inv_d[token_id].append(num_docs)  # aggiungo il docid alla lista dei docid in cui compare il termine
                inv_f[token_id].append(tf)  # aggiungo il tf alla lista dei tf in cui compare il termine
                lexicon[token][1] += 1  # incremento il df
                lexicon[token][2] += tf  # tf è quanto compare il termine nel documento
            doclen = len(tokens)
            doc_index.append((str(doc_id), doclen))
            total_dl += doclen
            num_docs += 1

            # Aggiorna la barra di avanzamento
            progress_bar.update(1)
    
    # Chiudere la barra di avanzamento
    progress_bar.close()

    # Compress the inv_d and inv_f lists
    """for term, (termid, df, _) in lexicon.items():
        # Compress the docids
        encoded_list = [vbyte_encode(x) for x in inv_d[termid]]
        concatenated_encoded = b''.join(encoded_list)
        assert inv_d[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
        inv_d[termid] = concatenated_encoded
        # Compress the frequencies
        encoded_list = [vbyte_encode(x) for x in inv_f[termid]]
        concatenated_encoded = b''.join(encoded_list)
        assert inv_f[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
        inv_f[termid] = concatenated_encoded"""

    stats = {
        'num_docs': num_docs,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [ ]:
from collections import Counter
from tqdm.auto import tqdm
import pandas as pd

@profile
def build_index_old(file_path, batchsize=10000):
    """
    Costruisce un indice invertito da un file TSV senza intestazione elaborato in batch,
    mostrando una barra di avanzamento completa.
    
    Args:
        file_path (str): Percorso al file TSV senza intestazione.
        batchsize (int): Numero di righe da elaborare per batch (default: 10000).
        
    Returns:
        tuple: lexicon (vocabolario), indici invertiti (docids, freqs), doc_index (documenti e lunghezze), stats (statistiche globali).
    """
    lexicon = {}
    doc_index = []
    inv_d, inv_f = {}, {}
    termid = 0

    num_docs = 0
    total_dl = 0

    # Calcolare il numero totale di righe/documenti
    with open(file_path, 'r', encoding='utf-8') as f:
        total_docs = sum(1 for _ in f)
    
    # Barra di avanzamento con totale definito
    progress_bar = tqdm(desc="Indexing", total=total_docs, unit="doc", dynamic_ncols=True)
    
    # Leggere il file TSV in batch
    for df in pd.read_csv(file_path, sep='\t', header=None, names=['doc_id', 'text'], chunksize=batchsize):
        for docid, (doc_id, doc_text) in enumerate(zip(df['doc_id'], df['text'])):
            tokens = preprocess_old(doc_text)
            token_tf = Counter(tokens)
            #doclen = sum(token_tf.values())
            for token, tf in token_tf.items():
                if token not in lexicon:
                    lexicon[token] = [termid, 0, 0]
                    inv_d[termid], inv_f[termid] = [], []
                    termid += 1
                token_id = lexicon[token][0]  # prendo il termid
                inv_d[token_id].append(num_docs)  # aggiungo il docid alla lista dei docid in cui compare il termine
                inv_f[token_id].append(tf)  # aggiungo il tf alla lista dei tf in cui compare il termine
                lexicon[token][1] += 1  # incremento il df
                lexicon[token][2] += tf  # tf è quanto compare il termine nel documento
            doclen = len(tokens)
            doc_index.append((str(doc_id), doclen))
            total_dl += doclen
            num_docs += 1

            # Aggiorna la barra di avanzamento
            progress_bar.update(1)
    
    # Chiudere la barra di avanzamento
    progress_bar.close()

    # Compress the inv_d and inv_f lists
    """for term, (termid, df, _) in lexicon.items():
        # Compress the docids
        encoded_list = [vbyte_encode(x) for x in inv_d[termid]]
        concatenated_encoded = b''.join(encoded_list)
        assert inv_d[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
        inv_d[termid] = concatenated_encoded
        # Compress the frequencies
        encoded_list = [vbyte_encode(x) for x in inv_f[termid]]
        concatenated_encoded = b''.join(encoded_list)
        assert inv_f[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
        inv_f[termid] = concatenated_encoded"""

    stats = {
        'num_docs': num_docs,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [ ]:
if chosen_collection == "vaswani":
    lex, inv, doc, stats = build_index("./vaswani.tsv")
elif chosen_collection == "MSMARCO": # requires approx 11 minutes
    lex, inv, doc, stats = build_index("../MSMARCO.tsv")

In [ ]:
expected_lexicon, expected_inv, expected_doc_index, expected_stats = build_index_old("../MSMARCO.tsv")

In [ ]:
try:
    assert lex == expected_lexicon, "Lexicon does not match expected"
    
    # Ordinare le liste prima di confrontarle
    # assert sorted(inv['docids']) == sorted(expected_inv['docids']), "Inverted document index does not match expected"
    # assert sorted(inv['freqs']) == sorted(expected_inv['freqs']), "Inverted frequencies do not match expected"
    
    assert inv['docids'] == expected_inv['docids'], "Inverted document index does not match expected"
    assert inv['freqs'] == expected_inv['freqs'], "Inverted frequencies do not match expected"
    
    
    
    # Ordinare l'indice dei documenti se è una lista
    assert doc == expected_doc_index, "Document index does not match expected"
    
    # Confronto per le statistiche, se sono dizionari o strutture simili
    assert stats == expected_stats, "Stats do not match expected"
    
except AssertionError as e:
    print(e)

In [ ]:
print(stats, "\n",expected_stats)

In [ ]:
def print_index(lex, inv, doc, stats): 
    print("**********************************************************************************\nFirst 5 elements of lexicon:")
    print(list(lex.items())[:5])

    print("\nFirst 5 elements of inv:\n\tFirst 5 elements of inverted index docids:")
    print("\t",{k: decode_concatenated_vbyte(v)[:5] for k, v in sorted(list(inv['docids'].items()))[:5]})
    print("\tFirst 5 elements of inverted index freqs:")
    print("\t",{k: decode_concatenated_vbyte(v)[:5] for k, v in list(inv['freqs'].items())[:5]})

    print("\nFirst 5 elements of document index:")
    print(doc[:5])

    print("\nStats:")
    print(stats,"\n**********************************************************************************")

print_index(lex, inv, doc, stats)
print_index(expected_lexicon, expected_inv, expected_doc_index, expected_stats)

In [ ]:
import cProfile
import pstats
import os

cProfile.run("build_index_old('./vaswani.tsv')", "output.prof")
p = pstats.Stats("output.prof")
p.sort_stats("cumtime").print_stats(20)  # Mostra le 20 funzioni più lente
os.remove("output.prof")